# Create a slice with PTP for precise time measurements

For running MF timestamp or OWL (one-way latency measurement) tools, slice nodes must meet prerequisites: 

+ Git and Dockerhub must be reachable 
+ Docker has to be running
+ PTP (Precision Time Protocol) clock must be running/

This notebook creates a 3-node slice and sets up all the the above

(Tested on 2023/08/18)

## Import the FABlib Library


In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

try:
    fablib = fablib_manager()
                     
    fablib.show_config()
except Exception as e:
    print(f"Exception: {e}")

## Create an Experiment Slice

The following creates three nodes, on different sites, with basic NICs connected to FABRIC's FABnetv4 internet. Sites can be changed, but make sure that the site is PTP-compatible. 

Current list of PTP-compatible sites:

- STAR
- MAX
- MICH
- MASS 
- UTAH
- NCSA
- UCSD
- FIU
- CLEM
- CERN

In [ ]:
slice_name = 'PTP_slice'

#[site1,site2,site3] = fablib.get_random_sites(count=2, 
#        avoid=["DALL","GPN","LBNL","RENC","SALT","TACC","UKY","WASH","NCSA","LOSA","GATECH","INDI","MAX", "MASS","NEWY","SRI","UCSD"])

# for faster execution, specify the sites (from the list above)
[site1, site2, site3] = ["STAR", "CERN", "CLEM"]

print(f"Sites: {site1}, {site2}, {site3}")

node1_name = 'Node1'
node2_name = 'Node2'
node3_name = 'Node3'


In [ ]:
#Create Slice. add_fabnet() automatically adds an L3 interface on each node and assigns an IP address.

slice = fablib.new_slice(name=slice_name)

# Node1
node1 = slice.add_node(name=node1_name, site=site1, image='docker_rocky_8')
node1.add_fabnet()

# Node2
node2 = slice.add_node(name=node2_name, site=site2, image='docker_rocky_8')
node2.add_fabnet()

# Node3
node3 = slice.add_node(name=node3_name, site=site3, image='docker_rocky_8')
node3.add_fabnet()

#Submit Slice Request
slice.submit();

## (Optional) Observe the Slice's Attributes

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    slice.show()
    slice.list_nodes()
    slice.list_networks()
    slice.list_interfaces()
except Exception as e:
    print(f"Exception: {e}")

<!-- ## Configure IP Addresses

### Get the Assigned Subnet

FABnetv4 networks are assigned a subnet and gateway by FABRIC.  You can get the subnet and available IPs from the FABlib objects.  -->

## Check connectivity via Experimenter's network

In [ ]:
try:
    node1 = slice.get_node(name=node1_name)   
    node3 = slice.get_node(name=node3_name)
    node3_addr = node3.get_interface(network_name=f'FABNET_IPv4_{node3.get_site()}').get_ip_addr()
    
    node1.execute('hostname')
    stdout, stderr = node1.execute(f'ping -c 5 {node3_addr}')  
    
except Exception as e:
    print(f"Exception: {e}")

# Prepare each node for time precision experiments

In [ ]:
nodes = slice.get_nodes()

## Enable NAT64

### Upload and Execute the NAT64 Script

We will use a NAT64 script, which configures an *IPv6* node so that it can access non-*IPv6* services. To view the script, click [here](./nat64.sh). This script sets up the [Public NAT64 Service](https://nat64.net) on your node.

For more details on how uploading and executing scripts works on FABRIC, click [here](../upload_and_execute/upload_and_execute.ipynb).

Based on https://nat64.net/
Note that NAT64 could go away at any minute, as it is a public service ran independently by Kasper Dupont, and is not affiliated with FABRIC.

In [ ]:
# Change this to all nodes

from ipaddress import ip_address, IPv6Address

for node in nodes:
    try:     
        # If the node is an IPv6 Node then configure NAT64
        if node.validIPAddress(node.get_management_ip()) == "IPv6":
            # needed to fix sudo unable to resolve error
            commands = """
            sudo echo -n "127.0.0.1 " | sudo cat - /etc/hostname  | sudo tee -a /etc/hosts;
            sudo echo -n "2a01:4f9:c010:3f02:64:0:8c52:7103       github.com\n"|sudo tee -a /etc/hosts;
            sudo echo -n "2a01:4f9:c010:3f02:64:0:8c52:7009       codeload.github.com\n"|sudo tee -a /etc/hosts;
            sudo echo -n "2a01:4f9:c010:3f02:64:0:b9c7:6e85       objects.githubusercontent.com\n"|sudo tee -a /etc/hosts;
            sudo echo -n "2600:1fa0:80b4:db49:34d9:6d1e::         ansible-galaxy.s3.amazonaws.com\n"|sudo tee -a /etc/hosts;
            sudo echo -n "2a01:4f9:c010:3f02:64:0:3455:9777       packages.confluent.io\n"|sudo tee -a /etc/hosts;
            """
            stdout, stderr = node.execute(commands, quiet=True)

        # Access non-IPv6 Services
        stdout, stderr = node.execute(f'sudo yum install -y -q git && git clone https://github.com/fabric-testbed/jupyter-examples.git')

        stdout, stderr = node.execute(f'ls jupyter-examples')

    except Exception as e:
        print(f"Exception: {e}")

## Set up PTP (Precision Time Protocol)

#### Clone MeasurementFramework Git repo Locally

In [ ]:
%%bash
rm -rf /tmp/MF;
git clone https://github.com/fabric-testbed/MeasurementFramework.git /tmp/MF;

#### Install and setup linux ptp package on all nodes 

In [ ]:
pre_requisites = f"""
`sudo apt-get update;sudo apt-get -y install ansible git` || `sudo dnf -y install epel-release ;sudo dnf -y install ansible git`
"""
ansible_instructions = f"""
cd /tmp/ansible;ansible-playbook --connection=local --inventory 127.0.0.1, --limit 127.0.0.1 playbook_fabric_experiment_ptp.yml;
"""

for node in nodes:
    print (f"Installing PTP on {node.get_name()}")
    node.upload_directory('/tmp/MF/instrumentize/ptp/ansible','/tmp/')
    node.execute(f"{pre_requisites}"\
                 f"{ansible_instructions}"\
                )
    print (f"Installation of PTP Completed on {node.get_name()}\n\n")

## Start Docker 

In [ ]:
for node in nodes:
    node.execute("sudo systemctl start docker")
    node.execute("sudo systemctl enable docker")
    node.execute("sudo usermod -aG docker rocky")
    
    print(f"\n Verify installtion... on {node.get_name()}")
    node.execute("docker --help")

# (Optional) Extend the slice (Add 14 days)

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    print(f"Lease End         : {slice.get_lease_end()}")
       
except Exception as e:
    print(f"Exception: {e}")

In [ ]:
import datetime

#Extend slice
end_date = (datetime.datetime.now().astimezone() + datetime.timedelta(days=14)).strftime("%Y-%m-%d %H:%M:%S %z")

try:
    slice = fablib.get_slice(name=slice_name)
    slice.renew(end_date)
    print(f"New lease end date : {slice.get_lease_end()}")
    
except Exception as e:
    print(f"Exception: {e}")

# Delete the Slice

In [ ]:
# try:
#     slice = fablib.get_slice(name=slice_name)
#     slice.delete()
# except Exception as e:
#     print(f"Exception: {e}")